<a href="https://colab.research.google.com/github/Ananassio/Data-Minig-Lab/blob/main/Week_2/Assingment_2.1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Assignment 2.1
### Scrape and Analyse

* API [https://beautiful-soup-4.readthedocs.io/en/latest/](https://beautiful-soup-4.readthedocs.io/en/latest/)

In [72]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np

#### Make float to Nums

In [248]:
help_dict = {
    'tousand':  1000,
    'million':  1000000,
    'billion':  1000000000,
    'trillion': 1000000000000,
    'None': 1,
}

def str_to_num(input):
  temp = pd.DataFrame(list(input.str.split(' ')))
  temp = temp.replace({1:help_dict})
  temp = temp.fillna(1)
  temp[0] = temp[0].str.lstrip('$')
  temp[0] = pd.to_numeric(temp[0])#.astype(np.float64)
  temp = temp[0]*temp[1]
  return temp

### Tasks
Scrape data from the website [http://www.nationmaster.com](http://www.nationmaster.com/), convert it into Pandas data frames and use pandas queries to answer the following questions: 

#### 1
Get the number of internet users per country, remove all NaN entries and return the top 10 countries with the highest absolute number of internet users. 

In [2]:
#use request to get data from URL
res = requests.get("http://www.nationmaster.com/country-info/stats/Media/Internet-users")
#parse data as HTML
soup = BeautifulSoup(res.content,'lxml')
#extract tables
table = soup.find_all('table')[0]
#convert table to pandas data frame
df = pd.read_html(str(table)) #returns list of dataframes (one for each table)

In [3]:
df = pd.DataFrame(df[0])
df

,#,COUNTRY,AMOUNT,DATE,GRAPH,HISTORY
0,1,China,389 million,2009,NaN,NaN
1,2,United States,245 million,2009,NaN,NaN
2,3,Japan,99.18 million,2009,NaN,NaN
3,NaN,Group of 7 countries (G7) average (profile),80.32 million,2009,NaN,NaN
4,4,Brazil,75.98 million,2009,NaN,NaN
...,...,...,...,...,...,...
244,214,Niue,1100,2009,NaN,NaN
245,=215,"Saint Helena, Ascension, and Tristan da Cunha",900,2009,NaN,NaN
246,=215,Saint Helena,900,2009,NaN,NaN
247,217,Tokelau,800,2008,NaN,NaN


In [4]:
df = df.fillna(0)
df = df.set_index('#').drop(0).reset_index()

In [5]:
df.head(10)

,#,COUNTRY,AMOUNT,DATE,GRAPH,HISTORY
0,1,China,389 million,2009,0.0,0.0
1,2,United States,245 million,2009,0.0,0.0
2,3,Japan,99.18 million,2009,0.0,0.0
3,4,Brazil,75.98 million,2009,0.0,0.0
4,5,Germany,65.12 million,2010,0.0,0.0
5,6,India,61.34 million,2009,0.0,0.0
6,7,Russia,59.7 million,2010,0.0,0.0
7,8,United Kingdom,51.44 million,2009,0.0,0.0
8,9,France,44.63 million,2010,0.0,0.0
9,10,Nigeria,43.99 million,2009,0.0,0.0


#### 2
Get the number of internet users per country, remove all NaN entries and return the top 10 countries with the highest number of internet users relative to the populutation. Hint: you need to scrape the population number from another page)

In [197]:
#use request to get data from URL
res2 = requests.get("https://www.nationmaster.com/country-info/stats/People/Population")
#parse data as HTML
soup2 = BeautifulSoup(res2.content,'lxml')
#extract tables
table2 = soup2.find_all('table')[0]
#convert table to pandas data frame
df2 = pd.read_html(str(table2)) #returns list of dataframes (one for each table)


In [198]:
df2 = pd.DataFrame(df2[0])
df2

,#,COUNTRY,AMOUNT,DATE,GRAPH,HISTORY
0,1,China,1.35 billion,2013,NaN,NaN
1,2,India,1.22 billion,2013,NaN,NaN
2,3,European Union,509.04 million,2012,NaN,NaN
3,NaN,South Asia average (profile),321.91 million,2013,NaN,NaN
4,4,United States,316.67 million,2013,NaN,NaN
...,...,...,...,...,...,...
296,=250,Coral Sea Islands,0.0,2013,NaN,NaN
297,=250,Clipperton Island,0.0,2013,NaN,NaN
298,=250,Wake Island,0.0,2013,NaN,NaN
299,=250,Spratly Islands,0.0,2013,NaN,NaN


In [199]:
df2 = df2.fillna(0)
df2 = df2.set_index('#').drop(0).reset_index()
df2 = df2.drop(2).reset_index().drop(['index','#','DATE','GRAPH','HISTORY'], axis=1)

In [200]:
df2 = df.merge(df2, how='inner', on='COUNTRY').drop(['#','DATE','GRAPH','HISTORY'], axis=1)
df2 = df2.rename(columns={"AMOUNT_x": "Internet User", "AMOUNT_y": "Population"})

In [201]:
df2

,COUNTRY,Internet User,Population
0,China,389 million,1.35 billion
1,United States,245 million,316.67 million
2,Japan,99.18 million,127.25 million
3,Brazil,75.98 million,201.01 million
4,Germany,65.12 million,81.15 million
...,...,...,...
213,Niue,1100,1229
214,"Saint Helena, Ascension, and Tristan da Cunha",900,7754
215,Saint Helena,900,7700
216,Tokelau,800,1353


In [204]:
df2['Internet User'] = str_to_num(df2['Internet User'])
df2['Population'] = str_to_num(df2['Population'])
df2['Internet User per citizen'] = df2['Internet User']/df2['Population']
df2

,COUNTRY,Internet User,Population,Internet User per citizen
0,China,389000000.0,1.350000e+09,0.288148
1,United States,245000000.0,3.166700e+08,0.773676
2,Japan,99180000.0,1.272500e+08,0.779411
3,Brazil,75980000.0,2.010100e+08,0.377991
4,Germany,65120000.0,8.115000e+07,0.802465
...,...,...,...,...
213,Niue,1100.0,1.229000e+03,0.895037
214,"Saint Helena, Ascension, and Tristan da Cunha",900.0,7.754000e+03,0.116069
215,Saint Helena,900.0,7.700000e+03,0.116883
216,Tokelau,800.0,1.353000e+03,0.591279


In [205]:
df2.sort_values(by='Internet User per citizen', ascending=False).head(10)

,COUNTRY,Internet User,Population,Internet User per citizen
128,Iceland,301600.0,315281.0,0.956607
54,Norway,4430000.0,4720000.0,0.938559
208,Falkland Islands (Islas Malvinas),2900.0,3140.0,0.923567
33,Sweden,8400000.0,9120000.0,0.921053
213,Niue,1100.0,1229.0,0.895037
27,Netherlands,14870000.0,16810000.0,0.884593
148,Saint Lucia,142900.0,162781.0,0.877867
47,Denmark,4750000.0,5560000.0,0.854317
51,Finland,4480000.0,5270000.0,0.850095
120,Luxembourg,424500.0,514862.0,0.824493


#### 3
Compute the correlation between the crime rate (murders per 100k) and the education level. Compare this to the correlation of crime rate and poverty (relative BIP). Hint: use pandas build in correlation function: [https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html)

In [ ]:
#use request to get data from URL
res3 = requests.get("https://www.nationmaster.com/country-info/stats/Crime/Murder-rate")
#parse data as HTML
soup3 = BeautifulSoup(res3.content,'lxml')
#extract tables
table3 = soup3.find_all('table')[0]
#convert table to pandas data frame
df3 = pd.read_html(str(table3)) #returns list of dataframes (one for each table)

In [ ]:
df3 = pd.DataFrame(df3[0])
df3

,COUNTRY,DESCRIPTION
0,Afghanistan,3.40
1,Albania,4.50
2,Algeria,0.64
3,Andorra,1.30
4,Antigua and Barbuda,8.50
...,...,...
146,Uruguay,5.80
147,Uzbekistan,3.20
148,Venezuela,49.00
149,Vietnam,1.90


correlation of crime rate and poverty (relative BIP)

In [254]:
#use request to get data from URL
res5 = requests.get("https://www.nationmaster.com/country-info/stats/Crime/Total-crimes-per-1000")
#parse data as HTML
soup5 = BeautifulSoup(res5.content,'lxml')
#extract tables
table5 = soup5.find_all('table')[0]
#convert table to pandas data frame
df5 = pd.read_html(str(table5)) #returns list of dataframes (one for each table)

#use request to get data from URL
res6 = requests.get("https://www.nationmaster.com/country-info/stats/Economy/GDP")
#parse data as HTML
soup6 = BeautifulSoup(res6.content,'lxml')
#extract tables
table6 = soup6.find_all('table')[0]
#convert table to pandas data frame
df6 = pd.read_html(str(table6)) #returns list of dataframes (one for each table)

In [255]:
df5 = pd.DataFrame(df5[0])
df5

,#,COUNTRY,AMOUNT,DATE,GRAPH
0,1.0,Iceland,209.520,2002,NaN
1,2.0,Sweden,138.350,2002,NaN
2,3.0,Dominica,112.550,2002,NaN
3,4.0,United Kingdom,109.960,2002,NaN
4,5.0,New Zealand,108.200,2002,NaN
...,...,...,...,...,...
83,77.0,Albania,1.620,2002,NaN
84,78.0,Yemen,1.300,2002,NaN
85,79.0,Philippines,1.060,2002,NaN
86,80.0,Burma,0.372,2002,NaN


In [256]:
df5 = df5.fillna(0)
df5 = df5.set_index('#').drop(0).reset_index()
df5 = df5.drop(2).reset_index().drop(['index','#','DATE','GRAPH',], axis=1)
df5

,COUNTRY,AMOUNT
0,Iceland,209.520
1,Sweden,138.350
2,United Kingdom,109.960
3,New Zealand,108.200
4,Finland,100.030
...,...,...
75,Albania,1.620
76,Yemen,1.300
77,Philippines,1.060
78,Burma,0.372


In [257]:
df6 = pd.DataFrame(df6[0])
df6

,#,COUNTRY,AMOUNT,DATE,GRAPH,HISTORY
0,1.0,European Union,$16.63 trillion,2012,NaN,NaN
1,2.0,United States,$15.68 trillion,2012,NaN,NaN
2,3.0,China,$8.36 trillion,2012,NaN,NaN
3,4.0,Japan,$5.96 trillion,2012,NaN,NaN
4,NaN,Group of 7 countries (G7) average (profile),$4.85 trillion,2012,NaN,NaN
...,...,...,...,...,...,...
232,200.0,Sao Tome and Principe,$263.73 million,2012,NaN,NaN
233,201.0,Palau,$228.42 million,2012,NaN,NaN
234,202.0,Marshall Islands,$186.90 million,2012,NaN,NaN
235,203.0,Kiribati,$175.71 million,2012,NaN,NaN


In [258]:
df6 = df6.fillna(0)
df6 = df6.set_index('#').drop(0).reset_index()
df6 = df6.drop(2).reset_index().drop(['index','#','DATE','GRAPH','HISTORY'], axis=1)
df6

,COUNTRY,AMOUNT
0,European Union,$16.63 trillion
1,United States,$15.68 trillion
2,Japan,$5.96 trillion
3,Germany,$3.40 trillion
4,France,$2.61 trillion
...,...,...
198,Sao Tome and Principe,$263.73 million
199,Palau,$228.42 million
200,Marshall Islands,$186.90 million
201,Kiribati,$175.71 million


In [259]:
df5 = df5.merge(df6, how='inner', on='COUNTRY')
df5 = df5.set_index('COUNTRY')
df5

,AMOUNT_x,AMOUNT_y
COUNTRY,,
Iceland,209.520,$13.66 billion
Sweden,138.350,$525.74 billion
United Kingdom,109.960,$2.44 trillion
New Zealand,108.200,$167.35 billion
Finland,100.030,$250.02 billion
...,...,...
India,1.640,$1.84 trillion
Albania,1.620,$13.12 billion
Yemen,1.300,$35.65 billion


In [260]:
df5['AMOUNT_y'] = str_to_num(df5['AMOUNT_y'])
df5

,AMOUNT_x,AMOUNT_y
COUNTRY,,
Iceland,209.520,NaN
Sweden,138.350,NaN
United Kingdom,109.960,NaN
New Zealand,108.200,NaN
Finland,100.030,NaN
...,...,...
India,1.640,NaN
Albania,1.620,NaN
Yemen,1.300,NaN


In [263]:
df5.corr()

,AMOUNT_x,AMOUNT_y
AMOUNT_x,1.0,NaN
AMOUNT_y,NaN,NaN


### REST API
#### Using data from [https://www.energidataservice.dk](https://www.energidataservice.dk) 

In [ ]:
import pandas as pd
import requests
from pandas import json_normalize

In [ ]:
#get data from an open energy data service provider
url = 'https://api.energidataservice.dk/dataset/NordpoolMarket?offset=0&sort=HourUTC%20DESC&timezone=dk'

response = requests.get(url)
dictr = response.json() #parse json to dict
recs = dictr['records'] 
df = json_normalize(recs) #flatten json files into data frame
df2 = df
df3 = df
df.head()

,HourUTC,HourDK,PriceArea,SpotPurchase,SpotSale,SpotPriceDKK,ElbasAveragePriceDKK,ElbasMaxPriceDKK,ElbasMinPriceDKK,SpotPriceEUR,ElbasAveragePriceEUR,ElbasMaxPriceEUR,ElbasMinPriceEUR
0,2021-08-22T21:00:00,2021-08-22T23:00:00,DK1,None,None,609.450012,None,None,None,81.949997,None,None,None
1,2021-08-22T21:00:00,2021-08-22T23:00:00,DK2,None,None,609.450012,None,None,None,81.949997,None,None,None
2,2021-08-22T20:00:00,2021-08-22T22:00:00,DK1,None,None,670.059998,None,None,None,90.099998,None,None,None
3,2021-08-22T20:00:00,2021-08-22T22:00:00,DK2,None,None,670.059998,None,None,None,90.099998,None,None,None
4,2021-08-22T19:00:00,2021-08-22T21:00:00,DK1,None,None,710.080017,None,None,None,95.480003,None,None,None


#### 4
Compute overview statistics (mean, variance, quantiles, counts,...) for all variables. Hint: there is a single pandas call to get this ...

In [ ]:
df.describe()

,SpotPriceDKK,SpotPriceEUR
count,100.000000,100.000000
mean,637.358407,85.702200
std,97.592793,13.122795
min,496.410004,66.750000
25%,568.479980,76.440002
50%,609.450012,81.949997
75%,708.510010,95.269997
max,834.200012,112.169998


#### 5 
Compute the average ***SpotSale*** by each day. 

In [ ]:
df = df.drop(['HourDK','PriceArea','SpotPurchase','SpotPriceDKK','ElbasAveragePriceDKK','ElbasMaxPriceDKK','ElbasMinPriceDKK','SpotPriceEUR','ElbasAveragePriceEUR','ElbasMaxPriceEUR','ElbasMinPriceEUR'], axis=1)
df['HourUTC'] = pd.to_datetime(df['HourUTC']).dt.date
df = df.groupby('HourUTC').mean()
df

,SpotSale
HourUTC,
2021-08-20,NaN
2021-08-21,NaN
2021-08-22,NaN


Da SportSale immer = NAN ist, wird noch mal SpotPriceEUR verwendet

In [ ]:
df2 = df2.drop(['HourDK','PriceArea','SpotPurchase','SpotPriceDKK','ElbasAveragePriceDKK','ElbasMaxPriceDKK','ElbasMinPriceDKK','SpotSale','ElbasAveragePriceEUR','ElbasMaxPriceEUR','ElbasMinPriceEUR'], axis=1)
df2['HourUTC'] = pd.to_datetime(df2['HourUTC']).dt.date
df2 = df2.groupby('HourUTC').mean()
df2

,SpotPriceEUR
HourUTC,
2021-08-20,103.600000
2021-08-21,90.050416
2021-08-22,77.704545


#### 6 
Compute the day with the highest variance in ***SpotPurchase***

Sportpurchase ist auch immer NAN, deshalb wird die Größte Varianz für SportPriceEUR verwendet

In [ ]:
df3 = df3.drop(['HourDK','PriceArea','SpotPurchase','SpotPriceDKK','ElbasAveragePriceDKK','ElbasMaxPriceDKK','ElbasMinPriceDKK','SpotSale','ElbasAveragePriceEUR','ElbasMaxPriceEUR','ElbasMinPriceEUR'], axis=1)
df3['HourUTC'] = pd.to_datetime(df2['HourUTC']).dt.date
df3MAX = df3.groupby('HourUTC').max()
df3MIN = df3.groupby('HourUTC').min()
df3MAX

,SpotPriceEUR
HourUTC,
2021-08-20,110.690002
2021-08-21,112.169998
2021-08-22,97.959999


In [ ]:
df3MIN

,SpotPriceEUR
HourUTC,
2021-08-20,95.199997
2021-08-21,76.589996
2021-08-22,66.750000


In [ ]:
(df3MAX-df3MIN).sort_values('SpotPriceEUR',ascending=False)

,SpotPriceEUR
HourUTC,
2021-08-21,35.580002
2021-08-22,31.209999
2021-08-20,15.490005
